In [78]:
import torch
import pandas as pd
import numpy as np
import torch.utils
import torchvision.transforms
from torch import nn
%matplotlib inline

In [79]:
#Loading house prices training and test dataset
hptrain = pd.read_csv("Datasets/hptrain.csv",index_col=0)
hptest = pd.read_csv("Datasets/hptest.csv",index_col=0)

In [80]:
#Transforming data into tensor data format from pandas' dataframe
X_ols_train = torch.tensor(hptrain[['LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'TotalBsmtSF']].values,requires_grad=True,dtype=torch.float32)
X_ols_train = nn.functional.normalize(X_ols_train)
Y_ols_train = torch.tensor(hptrain[['SalePrice']].values,requires_grad=True,dtype=torch.float32)


#Splitting data into iterable batches
olstensor = torch.utils.data.TensorDataset(X_ols_train,Y_ols_train)
olsdataloader = torch.utils.data.DataLoader(olstensor, batch_size=20,shuffle=True)

In [81]:
#Specifying the paramenters of our model
olsnet = nn.Sequential(nn.Linear(X_ols_train.size()[1],Y_ols_train.size()[1]))
olsnet[0].weight.data.normal_(0, 0.1)
olsnet[0].bias.data.normal_(0, 0.1)
olsloss = nn.MSELoss()
olstrainer = torch.optim.SGD(olsnet.parameters(),lr=1)

In [82]:
num_epochs = 10
olsnet.train()
for epoch in range(num_epochs):
    print('-------------------')
    print(f'{epoch+1}th epoch')
    print('===================')
    olstrainer.zero_grad()
    X, Y = next(iter(olsdataloader))
    output = olsnet(X)

    l = olsloss(output,Y)
    print(f'Loss ols:{l}')


    l.backward(retain_graph=True)

    olstrainer.step()
    print('Coefficients: ',olsnet[0].weight.data)


-------------------
1th epoch
Loss ols:39994900480.0
Coefficients:  tensor([[3.5908e+05, 2.4654e+02, 2.2933e+02, 7.5371e+03, 4.0230e+04]])
-------------------
2th epoch
Loss ols:313088081920.0
Coefficients:  tensor([[-7.2774e+05, -5.4098e+02, -5.3498e+02, -2.7362e+03, -7.8995e+04]])
-------------------
3th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
4th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
5th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
6th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
7th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
8th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
9th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
10th epoch
Loss ols:nan
Coefficients: 

In [83]:
'''--------------------------------------------------------'''
'''--------------------------------------------------------'''
'''--------------------------------------------------------'''

'--------------------------------------------------------'

In [84]:
#Loading in complex MNIST train and test datasets while transforming them to 64
trans = [torchvision.transforms.ToTensor()]
trans.insert(0, torchvision.transforms.Resize(64))
trans = torchvision.transforms.Compose(trans)


mnist_train = torchvision.datasets.FashionMNIST(
    root="../PyTorch_Basic/Datasets",train = True, transform=trans, download= True)
mnist_test = torchvision.datasets.FashionMNIST(
    root="../PyTorch_Basic/Datasets",train = False, transform=trans, download= True)

In [85]:
#Splitting our data into batch sizes
mnist_batch_size = 256

mnist_train_iter = torch.utils.data.DataLoader(mnist_train,mnist_batch_size,shuffle=True,num_workers=4)
mnist_test_iter = torch.utils.data.DataLoader(mnist_test,mnist_batch_size,shuffle=True,num_workers=4)

In [86]:
#Creating the model and initializing the weights
logit_net = nn.Sequential(nn.Flatten(),nn.Linear(784,10))

#Initializing weights
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

logit_net.apply(init_weights)

#Specifying cross entropy loss
logit_loss = nn.CrossEntropyLoss(reduction='none')

#Specifying Stochastic Gradient Descent trainer
logit_trainer = torch.optim.SGD(logit_net.parameters(), lr=0.1)